In [94]:
from sklearn.metrics import accuracy_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [95]:
class DRLModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DRLModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, output_dim)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [96]:
# Define a function to convert input data to tensors
def preprocess_input(input_data, topology_data):
    # Convert input data to tensors
    input_tensors = {}
    for topology, data in input_data.items():
        selected_middle_points = data['selected_middle_points']
        flow_demands = data['flow_demands']
        
        # Convert selected middle points to one-hot encoded tensor
        middle_points_tensor = torch.zeros(len(selected_middle_points))
        for i, point in enumerate(selected_middle_points):
            middle_points_tensor[i] = ord(point) - ord('A')  # Convert letter to one-hot index
        
        # Convert flow demands to tensor
        flow_demands_tensor = torch.tensor([[ord(src) - ord('A'), ord(dest) - ord('A'), demand] for src, dest, demand in flow_demands], dtype=torch.float32)
        
        # Get topology features
        topology_index = int(topology.split('_')[-1]) - 1
        topology_features = topology_data[topology_index]
        
        # Concatenate all features
        input_tensors[topology] = (torch.cat((middle_points_tensor, flow_demands_tensor.view(-1))), topology_features)
    
    return input_tensors

In [97]:
def preprocess_output(output_data):
    output_tensors = {}
    for topology, data in output_data.items():
        success_counts = [data[i]['success_count'] for i in range(1, 6)]  # Extract success counts
        output_tensors[topology] = torch.tensor(success_counts, dtype=torch.float32)
    return output_tensors

In [98]:
def train_drl_model(model, optimizer, num_epochs, input_data, output_data):
    for epoch in range(num_epochs):
        total_loss = 0.0
        for topology, (features, topology_features) in input_data.items():
            optimizer.zero_grad()
            predicted_output = model(torch.cat((features, topology_features)))
            loss = nn.MSELoss()(predicted_output, output_data[topology])
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
        if epoch % 100 == 0:
            print(f'Epoch [{epoch}/{num_epochs}], Total Loss: {total_loss}')

In [99]:
# Read topology files and extract topology features
def read_topology_file(filepath):
    with open(filepath, 'r') as file:
        content = file.readlines()
        nodes = [line.strip() for line in content[1:content.index('Edges:\n')]]
        edges = [line.split() for line in content[content.index('Edges:\n')+1:]]
    return nodes, edges

In [100]:
def extract_topology_features(nodes, edges):
    return torch.tensor([len(nodes), len(edges)], dtype=torch.float32)

In [101]:
# Load topology data
filepaths = [
    "M:/Group Project/ManojTopologies/topology_1.txt",
    "M:/Group Project/ManojTopologies/topology_2.txt",
    "M:/Group Project/ManojTopologies/topology_3.txt",
    "M:/Group Project/ManojTopologies/topology_4.txt",
    "M:/Group Project/ManojTopologies/topology_5.txt",    
]

topology_data = []
for filepath in filepaths:
    nodes, edges = read_topology_file(filepath)
    topology_data.append(extract_topology_features(nodes, edges))

# Define input and output data
input_data = {
    'Topology_1': {'selected_middle_points': ['C', 'B', 'D', 'A', 'F'], 'flow_demands': [('D', 'B', 13), ('E', 'C', 39), ('F', 'C', 37), ('B', 'E', 15), ('E', 'B', 12)]},
    'Topology_2': {'selected_middle_points': ['C', 'E', 'B', 'F', 'A'], 'flow_demands': [('E', 'D', 37), ('B', 'H', 47), ('I', 'F', 35), ('H', 'J', 41), ('B', 'E', 12)]},
    'Topology_3': {'selected_middle_points': ['B', 'A', 'D', 'F', 'H'], 'flow_demands': [('H', 'G', 10), ('B', 'H', 44), ('H', 'F', 30), ('A', 'D', 50), ('F', 'A', 25)]},
    'Topology_4': {'selected_middle_points': ['B', 'D', 'A', 'C', 'E'], 'flow_demands': [('E', 'G', 24), ('A', 'F', 40), ('D', 'E', 24), ('C', 'G', 44), ('F', 'E', 43)]},
    'Topology_5': {'selected_middle_points': ['A', 'F', 'B', 'C', 'H'], 'flow_demands': [('A', 'D', 34), ('C', 'B', 29), ('E', 'D', 30), ('E', 'B', 11), ('C', 'I', 31)]}
}

output_data = {
    'Topology_1': {1: {'success_count': 4}, 2: {'success_count': 3}, 3: {'success_count': 3}, 4: {'success_count': 3}, 5: {'success_count': 3}},
    'Topology_2': {1: {'success_count': 2}, 2: {'success_count': 2}, 3: {'success_count': 1}, 4: {'success_count': 1}, 5: {'success_count': 1}},
    'Topology_3': {1: {'success_count': 4}, 2: {'success_count': 3}, 3: {'success_count': 4}, 4: {'success_count': 4}, 5: {'success_count': 3}},
    'Topology_4': {1: {'success_count': 0}, 2: {'success_count': 0}, 3: {'success_count': 0}, 4: {'success_count': 0}, 5: {'success_count': 0}},
    'Topology_5': {1: {'success_count': 5}, 2: {'success_count': 5}, 3: {'success_count': 5}, 4: {'success_count': 5}, 5: {'success_count': 5}}
}

input_dim = len(input_data['Topology_1']['selected_middle_points']) + len(input_data['Topology_1']['flow_demands']) * 3 + 2  # Adding 2 for topology features
output_dim = 5  # Number of successful demand flows
model = DRLModel(input_dim, output_dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 1000

# Preprocess input and output data
input_tensors = preprocess_input(input_data, topology_data)
output_tensors = preprocess_output(output_data)

# Train the DRL model
train_drl_model(model, optimizer, num_epochs, input_tensors, output_tensors)

Epoch [0/1000], Total Loss: 45.73314583301544
Epoch [100/1000], Total Loss: 0.0004858516731474083
Epoch [200/1000], Total Loss: 3.84637038330915e-08
Epoch [300/1000], Total Loss: 2.5134672514419594e-12
Epoch [400/1000], Total Loss: 2.822742082461108e-13
Epoch [500/1000], Total Loss: 0.001513996598077938
Epoch [600/1000], Total Loss: 3.628430834869828e-13
Epoch [700/1000], Total Loss: 1.5816237573034147e-13
Epoch [800/1000], Total Loss: 3.887779084194129e-13
Epoch [900/1000], Total Loss: 5.048458388046129e-05


In [102]:
def preprocess_testing_input(input_data, topology_data):
    input_tensors = {}
    for topology, data in input_data.items():
        selected_middle_points = data['selected_middle_points']
        flow_demands = data['flow_demands']
        
        # Convert selected middle points to one-hot encoded tensor
        middle_points_tensor = torch.zeros(len(selected_middle_points))
        for i, point in enumerate(selected_middle_points):
            middle_points_tensor[i] = ord(point) - ord('A')  # Convert letter to one-hot index
        
        # Convert flow demands to tensor
        flow_demands_tensor = torch.tensor([[ord(src) - ord('A'), ord(dest) - ord('A'), demand] for src, dest, demand in flow_demands], dtype=torch.float32)
        
        # Get topology features
        topology_index = int(topology.split('_')[-1]) - 1
        topology_features = topology_data[topology_index]
        
        # Concatenate all features
        input_tensors[topology] = (torch.cat((middle_points_tensor, flow_demands_tensor.view(-1))), topology_features)
    
    return input_tensors

In [103]:
def predict_success_counts(model, input_data):
    success_counts = {}
    with torch.no_grad():
        for topology, (features, topology_features) in input_data.items():
            predicted_output = model(torch.cat((features, topology_features)))
            success_counts[topology] = predicted_output.argmax().item() + 1  # Select the action with maximum success count
    return success_counts

In [104]:
def load_topology_data(filepaths):
    topology_data = []
    for filepath in filepaths:
        nodes, edges = read_topology_file(filepath)
        topology_data.append(extract_topology_features(nodes, edges))
    return topology_data


In [105]:
testing_input_data = {
    'Topology_1': {'selected_middle_points': ['A', 'B', 'J', 'H', 'C'], 'flow_demands': [('G', 'R', 16), ('P', 'D', 46), ('J', 'F', 18), ('G', 'J', 34), ('C', 'F', 11)]},
    'Topology_2': {'selected_middle_points': ['C', 'B', 'A', 'F', 'G'], 'flow_demands': [('E', 'B', 35), ('C', 'I', 31), ('G', 'D', 50), ('R', 'P', 21), ('L', 'M', 23)]},
    'Topology_3': {'selected_middle_points': ['A', 'C', 'B', 'D', 'F'], 'flow_demands': [('G', 'F', 32), ('H', 'A', 23), ('C', 'D', 24), ('I', 'F', 44), ('E', 'A', 36)]},
    'Topology_4': {'selected_middle_points': ['B', 'C', 'H', 'E', 'F'], 'flow_demands': [('B', 'M', 22), ('E', 'M', 19), ('C', 'L', 18), ('D', 'C', 39), ('L', 'D', 34)]},
    'Topology_5': {'selected_middle_points': ['E', 'F', 'L', 'D', 'C'], 'flow_demands': [('G', 'E', 10), ('M', 'I', 20), ('E', 'H', 38), ('D', 'K', 29), ('K', 'F', 40)]}
}

filepathtest = [
    "M:/Group Project/Topology_new/Topology_new/topology_1.txt",
    "M:/Group Project/Topology_new/Topology_new/topology_2.txt",
    "M:/Group Project/Topology_new/Topology_new/topology_3.txt",
    "M:/Group Project/Topology_new/Topology_new/topology_4.txt",
    "M:/Group Project/Topology_new/Topology_new/topology_5.txt",  
]


testing_topology_data = load_topology_data(filepathtest)

# Preprocess testing input data
testing_input_tensors = preprocess_testing_input(testing_input_data, topology_data)

# Predict success counts for each topology
predicted_success_counts = predict_success_counts(model, testing_input_tensors)

# Print the predicted success counts for each topology
for topology, count in predicted_success_counts.items():
    print(f"Predicted optimal number of middle points for {topology}: {count}")

Predicted optimal number of middle points for Topology_1: 1
Predicted optimal number of middle points for Topology_2: 3
Predicted optimal number of middle points for Topology_3: 2
Predicted optimal number of middle points for Topology_4: 3
Predicted optimal number of middle points for Topology_5: 3


In [106]:
# Define ground truth success counts
ground_truth_success_counts = {
    'Topology_1': 3,
    'Topology_2': 2,
    'Topology_3': 3,
    'Topology_4': 3,
    'Topology_5': 2
}


In [107]:
ground_truth = [ground_truth_success_counts[topology] for topology in predicted_success_counts]


In [108]:
predicted = list(predicted_success_counts.values())
accuracy = accuracy_score(ground_truth, predicted)
f1 = f1_score(ground_truth, predicted, average='weighted')
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Accuracy: 0.2
F1 Score: 0.2
